# Tirer des stems au hasard pour un kalaba
- le tirage se fait sur la base d'un fichier Stems.yaml existant
- les stems sont remplacés par le tirage aléatoire
    - si le stem correspond à un nom propre, on adopte la prononciation standard du NP
    - si le stem d'origine n'a qu'un, deux ou trois phonèmes, on conserve la même longueur
    - si le stem fait au moins 4 phonèmes de long, on tire un radical CVCVC

In [48]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [49]:
from string import translate, maketrans

In [50]:
home = expanduser("~")
annee="20"
def getSerie(nomKalaba):
    return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s-%s/"%(annee,nomKalaba)

serie=getSerie("K5")

nomsPropres={"Kaleb":"kaleb","Mahira":"maira","Violette":"vjolet","Nabil":"nabil","Katisha":"katiSa","Nicole":"nikol"}

with open(serie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.load(stream)

In [51]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [52]:
lexique=getLeaves(prevStems)
nextLexique=getLeaves(prevStems)

In [53]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [54]:
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)

In [55]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [56]:
consonnes

'ptkbdgmnNfTsSDvzZrljw'

In [57]:
def randomRadical(radical):
    result=""
    consRadical=rd.sample(consonnes,3)
    vocRadical=rd.sample(voyelles,2)
    if len(radical)>3:
        result=consRadical[0]+vocRadical[0]+consRadical[1]+vocRadical[1]+consRadical[2]
    elif len(radical)==3:
        result=consRadical[0]+vocRadical[0]+consRadical[1]
    elif len(radical)==2:
        result=consRadical[0]+vocRadical[0]
    else:
        result=consRadical[0]
    return result

In [58]:
randomRadical("term")

'lesuj'

In [59]:
racineChanges={}
for mot in lexique:
    if mot in nextLexique:
        vedette=mot.split(",")[0].strip()
        if vedette in nomsPropres:
            racineChanges[nextLexique[mot]]=nomsPropres[vedette]
        else:
            racineChanges[nextLexique[mot]]=randomRadical(lexique[mot])
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

In [60]:
racineChanges

{'Napel': 'DeSim',
 'Sakap': 'Subaf',
 'Sarif': 'NenaD',
 'SaweD': 'Zoker',
 'Sawel': 'godan',
 'Sur': 'Taz',
 'Tetak': 'tajor',
 'Tijon': 'wanup',
 'Tirel': 'tadus',
 'TomuN': 'jaZug',
 'Toren': 'pufir',
 'Zajem': 'milap',
 'b': 'n',
 'baner': 'mukot',
 'bawak': 'baNuZ',
 'beliS': 'medip',
 'beliz': 'bogun',
 'beran': 'wamib',
 'berin': 'Semap',
 'beron': 'Sesub',
 'berub': 'zivaD',
 'biN': 'weT',
 'bit': 'DoN',
 'buwaN': 'muZeg',
 'dalat': 'taSim',
 'davos': 'bupeZ',
 'deZat': 'jiNuk',
 'dekat': 'riwaf',
 'deraN': 'jufoN',
 'derogo': 'Sokig',
 'doTar': 'tuvij',
 'feriT': 'setum',
 'fet': 'zos',
 'for': 'jel',
 'gav': 'DeT',
 'geZoj': 'winoz',
 'gemov': 'maluD',
 'gereg': 'geNim',
 'gigit': 'wuzam',
 'girit': 'nomiD',
 'gozet': 'tariD',
 'juron': 'Tewig',
 'k': 'r',
 'kaSoN': 'teNon',
 'kadiw': 'NuZar',
 'kaleb': 'kaleb',
 'kasiD': 'zitap',
 'katiSa': 'katiSa',
 'kejar': 'ZaveT',
 'keleN': 'fesil',
 'kerat': 'Dipof',
 'kiNol': 'kaSod',
 'koSaN': 'sabin',
 'koSoN': 'TaziD',
 'koj': 'ra

In [61]:
newStems=replace_keys(prevStems,racineChanges)

In [62]:
with open(serie+'randomStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [63]:
with open(serie+"randomStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [64]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  A1:
    Sesub: [différent, différents, différente, différentes]
    Zubow: [bas, basse, basses]
    geNim: [gros, grosse, grosses]
    jaZug: [courageux, courageuse, courageuses]
    jufoN: [trois]
    kazuw: [noir, noire, noirs, noires]
    milap: [grand, grande, grands, grandes]
    mojip: [profond, profonds, profonde, profondes]
    moreT: [rouge, rouges]
    wanup: [suivant, suivants, suivante, suivantes]
  A2:
    Sokig: [terrible, terribles]
    forut: [jaune, jaunes]
    godan: [maigre, maigres]
    nomiD: [blanc, blanche, blancs, blanches]
    polaS: [furieux, furieuse, furieuses]
    sajun: [quatre]
    sorad: [petit, petite, petits, petites]
    vikuj: [effrayé, effrayés, effrayée, effrayées]
    wamib: [blessé, blessés, blessée, blessées]
DET:
  n: [DEM, ce, cet, cette, ces]
  r: [IND, un, une, des, dE]
  t: [DEF, le, la, les, l]
NOM:
  N1:
    A:
      miNaz: [chasseur, chasseurs]
      rofuj: [balai, balais]
      soTup: [forêt, forêts]
      vokef: [soeur, soeurs, 

In [65]:
with open(serie+'randomStems.yaml', 'w') as output:
    output.write(yamlText)